# Beneficiários

In [1]:
import io
import tempfile
import requests

import pandas as pd
import numpy as np

import src.info_beneficiarios as ib 

In [2]:
# donwload dos códigos ANS de Hapvida e GNDI

RI_URL = 'https://api.mziq.com/mzfilemanager/v2/d/6bbd1770-f9f4-44e8-a1b1-d26b7585eec1/6d20a378-c816-6a0a-ad88-d557761215bd?origin=1'

def ans_codes(url, empresa):
    response = requests.get(url)

    temp_file = tempfile.NamedTemporaryFile(delete=False)
    temp_file.write(response.content)

    if empresa == 'HAPVIDA':
        df = pd.read_excel(temp_file.name, sheet_name='Códigos ANS_ANS Code', skiprows=1, nrows=20, usecols=[0,1,2,3,4])
        df.columns = ['CD_ANS', 'EMPRESA', 'GRUPO', 'ASSINATURA', 'FECHAMENTO']

    elif empresa == 'GNDI':
        df = pd.read_excel(temp_file.name, sheet_name='Códigos ANS_ANS Code', skiprows=1, nrows=20, usecols=[6, 7, 8, 9])
        df.columns = ['CD_ANS', 'EMPRESA', 'ASSINATURA', 'FECHAMENTO']

    else:
        raise ValueError("Invalid empresa value. Must be 'HAPVIDA' or 'GNDI'.")

    # df['CD_ANS'] = df['CD_ANS'].astype(object)
    
    temp_file.close()
    
    return df

In [3]:
# download ANS codes

hapvida = ans_codes(RI_URL, 'HAPVIDA')
intermedica = ans_codes(RI_URL, 'GNDI')

combined_codes = pd.concat([hapvida.iloc[:, :2], intermedica.iloc[:, :2]])
combined_codes = combined_codes.dropna(subset=['CD_ANS'])['CD_ANS'].astype('int32')

combined_codes.head(10)


0    368253
1    415847
2    351091
3    302091
4    323811
5    309338
6    339539
7    333328
8    365319
9    405604
Name: CD_ANS, dtype: int32

In [4]:
# download SIB data

url = 'https://dadosabertos.ans.gov.br/FTP/PDA/informacoes_consolidadas_de_beneficiarios/202310'

data = ib.download_data(url)

In [5]:
data.head()

,#ID_CMPT_MOVEL,CD_OPERADORA,NM_RAZAO_SOCIAL,NR_CNPJ,MODALIDADE_OPERADORA,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,TP_SEXO,DE_FAIXA_ETARIA,...,TP_VIGENCIA_PLANO,DE_CONTRATACAO_PLANO,DE_SEGMENTACAO_PLANO,DE_ABRG_GEOGRAFICA_PLANO,COBERTURA_ASSIST_PLAN,TIPO_VINCULO,QT_BENEFICIARIO_ATIVO,QT_BENEFICIARIO_ADERIDO,QT_BENEFICIARIO_CANCELADO,DT_CARGA
0,202310,326305,AMIL ASSISTÊNCIA MÉDICA INTERNACIONAL S.A.,29309127000179,MEDICINA DE GRUPO,AC,120040,RIO BRANCO,F,41 a 45 anos,...,P,COLETIVO POR ADESÃO,AMBULATORIAL + HOSPITALAR COM OBSTETRÍCIA,NACIONAL,Médico-hospitalar,DEPENDENTE,1,0,0,01/12/2023
1,202310,368253,HAPVIDA ASSISTENCIA MEDICA S.A.,63554067000198,MEDICINA DE GRUPO,AC,120040,RIO BRANCO,F,31 a 35 anos,...,P,COLETIVO EMPRESARIAL,ODONTOLÓGICO,GRUPO DE ESTADOS,Odontológico,TITULAR,1,0,0,01/12/2023
2,202310,374440,PREVIDENT ASSISTÊNCIA ODONTOLÓGICA S.A,56269913000162,ODONTOLOGIA DE GRUPO,AC,120080,PORTO ACRE,M,61 ou mais,...,P,INDIVIDUAL OU FAMILIAR,ODONTOLÓGICO,NACIONAL,Odontológico,TITULAR,1,0,0,01/12/2023
3,202310,419419,BRASILDENTAL OPERADORA DE PLANOS ODONTOLÓGICOS...,19962272000109,ODONTOLOGIA DE GRUPO,AC,120010,BRASILEIA,M,41 a 45 anos,...,P,COLETIVO EMPRESARIAL,ODONTOLÓGICO,NACIONAL,Odontológico,TITULAR,2,0,0,01/12/2023
4,202310,301949,ODONTOPREV S/A,58119199000151,ODONTOLOGIA DE GRUPO,AC,120040,RIO BRANCO,F,61 ou mais,...,P,COLETIVO EMPRESARIAL,ODONTOLÓGICO,NACIONAL,Odontológico,TITULAR,5,0,0,01/12/2023


In [6]:
filtered_data = data.loc[data['COBERTURA_ASSIST_PLAN'] == 'Médico-hospitalar', :]

filtered_data['CAT_OPERADORA'] = np.where(filtered_data['CD_OPERADORA'].isin(combined_codes), 'HAPVIDA', 'OTHER')

grouped_data = filtered_data.groupby('CAT_OPERADORA')['QT_BENEFICIARIO_ATIVO'].sum()

percentage_data = grouped_data / grouped_data.sum() * 100

percentage_data


/tmp/ipykernel_25128/3217502003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['CAT_OPERADORA'] = np.where(filtered_data['CD_OPERADORA'].isin(combined_codes), 'HAPVIDA', 'OTHER')


CAT_OPERADORA
HAPVIDA     8.999424
OTHER      91.000576
Name: QT_BENEFICIARIO_ATIVO, dtype: float64